# Exercise 6: Vanilla Policy Gradient (VPG)

In [ ]:
import numpy as np
import os
import tensorflow as tf

from unityagents import UnityEnvironment

### Hyperparameters

In [ ]:
y = 0.99 # Discount rate.
total_episodes = 20000 #Set total number of episodes to train agent on.
update_frequency = 10 # How many episodes before updating model.
summary_freq = 50 # How often to display summary statistics.
learning_rate = 1e-3 # Agent learning rate.
hidden_units = 128 # Number of units in hidden layer.
summary_path = './summaries/vpg' # Where to save summary statistics.
model_path = './models/vpg' # Where to save model checkpoints.
load_model = False # Whether to load a saved model.
train_model = True # Whether to train the model.

### Load the Unity environment

In [ ]:
env = UnityEnvironment("envs./3DBall", worker_id=5)
default_brain = env.brain_names[0]

### Examine state space

In [ ]:
brains = env.reset()
agent_brain = brain[default_brain]
print(agent_brain.vector_observations)

State (s) is a vector whose values corresponds to:
* Platform Z rotation
* Platform X rotation
* Ball X position
* Ball Y position
* Ball Z position
* Ball X velocity
* Ball Y velocity
* Ball Z velocity

### The Actor Critic Agent

In [ ]:
class VPGAgent():
    def __init__(self, lr, s_size,a_size,h_size):
        # These lines established the feed-forward part of the network. 
        # The agent takes a state and produces an action.
        self.state_in= tf.placeholder(shape=[None,s_size],dtype=tf.float32)
        self.batch_size = tf.placeholder(shape=None, dtype=tf.int32)
        hidden = tf.layers.dense(self.state_in, h_size, use_buas=False, activation=tf.nn.elu)
        self.policy = tf.layers.dense(hidden, a_size, activation=tf.nn.softmax, use_bias=False)
        self.sampled_action = tf.multinomial(self.policy, 1)
        
        # The randomness in the action probabilities.
        self.entropy = -tf.reduce_sum(self.policy * tf.log(self.policy + 1e-10), axis=1)
        
        # The next six lines establish the training proceedure. 
        # We feed the reward and chosen action into the network
        # to compute the loss, and use it to update the network.
        self.reward_holder = tf.placeholder(shape=[None],dtype=tf.float32)
        self.action_holder = tf.placeholder(shape=[None],dtype=tf.int32)
        self.actions = tf.one_hot(self.action_holder, a_size)
        
        self.responsible_outputs = tf.reduce_sum(self.output * self.actions, axis=1)
        self.advantage = self.reward_holder
        self.loss = -tf.reduce_mean(tf.log(self.responsible_outputs)*self.advantage)
        
        # This is used to collect gradient for each tensor in the network.
        tvars = tf.trainable_variables()
        self.gradient_holders = []
        for idx,var in enumerate(tvars):
            placeholder = tf.placeholder(tf.float32,name=str(idx)+'_holder')
            self.gradient_holders.append(placeholder)
        
        self.gradients = tf.gradients(self.loss,tvars)
        
        # Here we apply the gradients to update the network. 
        optimizer = tf.train.AdamOptimizer(learning_rate=lr)
        self.update_batch = optimizer.apply_gradients(zip(self.gradient_holders,tvars))

### Training the Agent

In [ ]:
# Function to take list of rewards and discount factor
# and produce discounted sum of future rewards.
def discount_rewards(r, gamma):
    discounted_r = np.zeros_like(r)
    running_add = 0
    for t in reversed(range(0, r.size)):
        running_add = running_add * gamma + r[t]
        discounted_r[t] = running_add
    return discounted_r

In [ ]:
if not os.path.exists(summary_path):
    os.makedirs(summary_path)

if not os.path.exists(model_path):
    os.makedirs(model_path)

# Create our tensorflow agent
tf.reset_default_graph()
space_size = env.brains[default_brain].vector_observation_space_size
action_size = env.brains[default_brain].vector_action_space_size
myAgent = VPGAgent(lr=learning_rate, s_size=space_size, 
                   a_size=action_size, h_size=hidden_units) 

# Start an interactive TensorFlow session.
sess = tf.InteractiveSession()
init = tf.global_variables_initializer()
sess.run(init)
summary_writer = tf.summary.FileWriter(summary_path)
saver = tf.train.Saver()

# Optionally load the model.
if load_model:
    print('Loading Model...')
    ckpt = tf.train.get_checkpoint_state(model_path)
    saver.restore(sess,ckpt.model_checkpoint_path)
    
# Create buffer for storing gradients.
gradBuffer = sess.run(tf.trainable_variables())
for ix,grad in enumerate(gradBuffer):
    gradBuffer[ix] = grad * 0
    
# Create variables which will be used throughout training.
total_reward = []
total_length = []
total_entropy = []

# Start training loop
for i in range(total_episodes):
    brain = env.reset()[default_brain]
    state = brain.vector_observations[0]
    episode_steps = 0
    running_reward = 0
    done = False
    ep_history = []
    episode_entropy = []
    while not done:
        episode_steps += 1
        
        # Probabilistically pick an action given our network outputs.
        action, entropy = sess.run([myAgent.sampled_action, myAgent.entropy], 
                          feed_dict={myAgent.state_in:[state], myAgent.batch_size: 1})
        
        # Take action in environment.
        brain_info = env.step(action)[default_brain]
        
        # Collect resulting reward and new observation.
        state_1 = brain_info.vector_observations[0]
        reward = brain_info.rewards[0]
        done = brain_info.local_done[0]
        ep_history.append([state, action, reward, state_1])
        episode_entropy.append(entropy)
        
        state = state_1
        running_reward += reward
        
        if done and train_model:
            # Collect gradients for episode.
            ep_history = np.array(ep_history)
            ep_history[:,2] = discount_rewards(ep_history[:,2], y)
            feed_dict={myAgent.reward_holder:ep_history[:,2],
                    myAgent.action_holder:ep_history[:,1],
                    myAgent.state_in:np.vstack(ep_history[:,0]),
                    myAgent.batch_size: len(ep_history)}
            p_loss, grads = sess.run([myAgent.loss, myAgent.gradients], feed_dict=feed_dict)
            for idx,grad in enumerate(grads):
                gradBuffer[idx] += grad

            # Apply gradients for past 'n' episodes.
            if i % update_frequency == 0 and i != 0:
                feed_dict = dictionary = dict(zip(myAgent.gradient_holders, gradBuffer))
                _ = sess.run(myAgent.update_batch, feed_dict=feed_dict)
                for ix,grad in enumerate(gradBuffer):
                    gradBuffer[ix] = grad * 0
    total_entropy.append(np.mean(episode_entropy))
    total_reward.append(running_reward)
    total_length.append(episode_steps)

    # Update our running tally of scores and save information to Tensorboard.
    if i % summary_freq == 0 and i != 0:
        summary = tf.Summary()
        summary.value.add(tag='Info/Mean Reward', simple_value=float(np.mean(total_reward[-summary_freq:])))
        summary.value.add(tag='Info/Entropy', simple_value=float(np.mean(total_entropy[-summary_freq:])))
        summary.value.add(tag='Info/Policy Loss', simple_value=float(p_loss))
        summary_writer.add_summary(summary, i)
        summary_writer.flush()
        print("Episode: {}, Epsilon: {}, Mean Reward: {}".format(str(i), str(e), str(round(np.mean(total_reward[-summary_freq:]), 3))))
    # Save agent's model
    if i % 1000 == 0 and i != 0:
        saver.save(sess, model_path+'/model-'+str(i)+'.cptk')
        print("Saved Model")
env.close()

In [ ]:
env.close()